In [27]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.palettes import Spectral3, Spectral7, Spectral8, Spectral10, brewer
from bokeh.models import DatetimeTickFormatter, ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
from datetime import datetime as dt

# Python Visualization Landscape

There are many different visualization possibilities when using Python. In this chapter we will only focus on the ones based on JavaScript however, we will quickly describe each of them so that, on the future, you can have a better overview and focus on the ones that best fit your purpuse. 

<img src="./Figures/PythonVizLandscape.pdf">

Figure adapted from Jake VanderPlas and [PyViz](https://pyviz.org/overviews/index.html).

- **[Matplotlib](https://matplotlib.org)** was first released in 2003 (J.D. Hunter, Matplotlib: A 2D graphics environment, 2007) and is the core tool for data visualization in Python. Many libraries have been built using Matplotlib as backbone. This library has a similar design like Matlab which was the main idea so that people could easily switch from Matlab to Python. Matplotlib has a Matlab emulation environment called PyLab, which is a simple wrapper of the matplotlib API. Matplotlib supports many rendering back-ends which allows you save the plots in different formats like png, svg, pdf, etc. What is great about this library is that you can reproduce almost any plot you want. This library has been used for over almost two decades which makes it a very well tested and standard tool. Many visualizations libraries keep Matplotlib as a versatile and well tested back-end but, innovate providing new and easier ways to generate the plots. 

    - [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) has built-in plotting functions which are usually in the form of ``df.plot.scatter()`` . It provides other sophisticated tools than the ones already in Matplotlib in order to show labaled data in a figure.
    - [Seaborn](https://seaborn.pydata.org) has the main objective to perform complex statistical visualizations in an easier way. This library focuses on visualizing models and statistical datasets. 

- **JavaScript** based libraries build a new API that produces a plot serialization (often a JSON file) that can be displayed in the browser (for example, in Jupyter Notebooks). Through a JavaScript code the serialization can be read and rendered into plots. 
    - [Plotly](https://plotly.com) is an interactive browser-based graphing library for Python. It allows web and view interactivity, support multi-language (you can find this library on Julia and R) and has 3D plotting and animation capabilities. 
    - [Bokeh](http://bokeh.org) is the library we will be discussing in more detial. It is one of the most used interactive data visualization libraries. We will see some examples in this notebook. 
        - [Holoviews](http://holoviews.org) can use both bokeh or Matplotlib or Ploty to render the plots. We will also see in more detail this library in the next notebook (NB-4-Holoviews.ipynb). 
        - [Datashader](\url{https://datashader.org/index.html#}) is a fast server-side engine for dynamics data aggregation tht allows quick representations of data sets. Instead of delivering data points for rendering, Datashader pre-aggregates the data and delivers pixels (like bitmaps) for the computer to render. So the creation of an image is broken into a series of explicit seps which allow the computation of intermediate representations. 
- **[D3.js](https://d3js.org)** is a JavaScript library for manipulation and streamlines of objects on a web page. This library allows you to bind arbitrary data to a Document Object Model (DOM) and then apply data-driven transformations to the document. It is very fast which makes it very useful for large datasets. The main drawback is that this language is very verbose.
    - [VEGA](https://vega.github.io/vega/about/vega-and-d3/) provides a higher level visualization specification language on top of D3. VEGA has detailed declarative specification for visualizations built in D3. 
    - [Vega-Lite](https://vega.github.io/vega-lite/) allows more concise ways to create the codes for plots. It provides declarative and concise JSON syntax to create a hole range of data representations and statistical representations. But if you want to benefit from all the awesome features from these packages but using Python programming language then Altair is the way to go. 
    - [Altair](https://altair-viz.github.io) is a declarative statistical visualization library for Python which is based on Vega and Vega-Lite.  


# Bokeh Plotting

[Bokeh](https://bokeh.org) is an interactive, sharable, open source, flexible and powerful visualization library for modern web browsers. Bokeh is the tool you need to easily create interactive plots that you can later save or share via a web browser. 

Bokeh has 2 interface levels for plotting depending on what you are looking for when creating a figure:

1. ``bokeh.models`` $\rightarrow$ Low-level interface that provides the most flexibility to application developers (to better understand the structure of this interface, visit their web [explanation](https://docs.bokeh.org/en/latest/docs/reference/models.html)). 
2. ``bokeh.plotting`` $\rightarrow$ Higher-level interface centerd around composing visual glyphs. 

In this chapter we will only cover ``bokeh.plotting`` since when performing an image analysis task, the last steps of the workflow usually require to compose visual representations of the results. However we do give an example on how to use some functions from the ``bokeh.models``. The [user guide](https://docs.bokeh.org/en/latest/docs/user_guide.html) from the Bokeh web page also focuses on this second interface level of plotting so make sure you go through their great examples. 

## Steps to create a figure with bokeh.plotting

In order to create a figure with ``bokeh.plotting``, there are 5 basic steps (that can later become customized and therefore more complicated) in order to generate a plot:

[ 1 ] Prepare the data you want to plot. This is something we have discussed already in the chapter which involves for example, importing your data into pandas, creating a DataFrame that best describes your table-like data, tidy this DataFrame with the ``pd.melt()``function, use the split-apply-combine strategy to generate some meaningful analysis and choose the analyzed data you are willing to represent as a plot. 

[ 2 ] Specify where you want Bokeh to generate the figure. Mainly, you have 2 options: (1) ``output_file(name_file.html)`` will generate a new output file "name_file.html" and a browser automatically opens a new tab to display it. (2) ``output_notebook()`` to generate output in notebook cells.

[ 3 ] Create a new Figure for plotting by calling ``figure()``. This creates a plot with typical default options and easy customization of title, tools, and axes labels: ``figure(tools, title, y_axis_label, x_axis_label, y_range, x_range, y_axis_type, x_axis_type)``

[ 4 ] Define which type of plot you want by adding the renderers: ``line()``, ``dot()``, ``circle()``, ``vbar()``, ``hbar()``, etc. (For more examples of renderers, visit the [link](https://docs.bokeh.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.line)).

[ 5 ] Ask Bokeh to ``show()`` or ``save()`` the results. These functions save the plot to an HTML file and optionally display it in a browser.

In [2]:
# Prepare the data you want to plot
df1 = pd.DataFrame({'A':np.random.random(60), 'B':np.random.uniform(0,10,60)})
df2 = pd.DataFrame({'A':np.random.random(60), 'B':np.random.uniform(0,10,60)})

# Choose the columns you want to plot
x = 'A'
y = 'B'

# Specify where to output the figure
output_notebook()

# Specify the tools if you want to add or remove any
TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select"

# Create a figure
p = figure(width=400, plot_height=300, tools=TOOLS)

# Define the renderer
p.circle(x=x, y=y, source=df1, size=15, color=Spectral10[5], line_color='black')
p.circle(x=x, y=y, source=df2, size=15, color=Spectral10[1], line_color='black')

# Show the result
show(p)

Loading BokehJS ...

We can also add more levels of interactivity like for example using the legend.

In [3]:
# Prepare the data you want to plot
df1 = pd.DataFrame({'A':np.random.random(60), 'B':np.random.uniform(0,10,60)})
df2 = pd.DataFrame({'A':np.random.random(60), 'B':np.random.uniform(0,10,60)})

# Choose the columns you want to plot
x = 'A'
y = 'B'

# Specify where to output the figure
output_notebook()

# Specify the tools if you want to add or remove any
TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select"

# Create a figure
p = figure(width=400, plot_height=300, tools=TOOLS)

# Define the renderer
p.circle(x=x, y=y, source=df1, size=15, color=Spectral10[5], line_color='black', legend_label='Data 1')
p.circle(x=x, y=y, source=df2, size=15, color=Spectral10[1], line_color='black', legend_label='Data 2')

# Interactive legend
p.legend.location = "top_left"
p.legend.click_policy="hide"

# Show the result
show(p)

Loading BokehJS ...

## Example bokeh plots using DataFrames (from csv files)

In the following lines, we use the data for Pulse vs Concentration & Treatme vs Concentration which we already showed in section 3.5 Split-Apply-Combine of the chapter. 

As we mentioned in the chapter, we will first convert the data into a tidy format and then we will group the data according to one of the categories. As you will see, this will make your plotting much easier and also, your code will be easier to read and to share. 

In [28]:
# Upload some data
df = pd.read_csv('Data/PulseVsConcentration.csv')
df

,Concentration,5 min,10 min,20 min,30 min
0,500 um,2.3,9.2,12.5,16.9
1,100 um,5.4,9.9,13.3,17.0
2,20 um,3.2,9.8,13.5,17.4
3,10 um,4.8,9.2,14.2,17.7


In [5]:
# Convert into a tidy format using the melt function
df_melt = pd.melt(df, id_vars='Concentration', var_name='Pulse Duration', value_name='Result')
df_melt.head()

,Concentration,Pulse Duration,Result
0,500 um,5 min,2.3
1,100 um,5 min,5.4
2,20 um,5 min,3.2
3,10 um,5 min,4.8
4,500 um,10 min,9.2


In [6]:
# Group the data according to the Concentration category using the groupby method
g1 = df_melt.groupby('Concentration')
g1.get_group('500 um')

,Concentration,Pulse Duration,Result
0,500 um,5 min,2.3
4,500 um,10 min,9.2
8,500 um,20 min,12.5
12,500 um,30 min,16.9


Once we have arranged our data, we will try to plot all we can with it! The following cells show examples of line plots, scatter plots, histograms and box/violin plots. These are some of the most used types of plots which is why we will learn how to create some basic figures with these graphs. 

## Line Plots

In [30]:
# Choose the data you want to plot
x = [5, 10, 20, 30]
y1 = g1.get_group('500 um')['Result']
y2 = g1.get_group('100 um')['Result']
y3 = g1.get_group('20 um')['Result']
y4 = g1.get_group('10 um')['Result']

# Create the figure
p = figure(plot_width=500, plot_height=300)


# add a circle renderer with a size, color, and alpha
p.circle(x=x, y=y1, size=20, color=Spectral10[0], alpha=0.5, line_color='black', legend_label='500 uM')
p.line(x=x, y=y1, line_width=2, color=Spectral10[0])

p.circle(x=x, y=y2, size=20, color=Spectral10[1], alpha=0.5, line_color='black', legend_label='100 uM')
p.line(x=x, y=y2, line_width=2,color=Spectral10[1])

p.circle(x=x, y=y3, size=20, color=Spectral10[2], alpha=0.5, line_color='black', legend_label='20 uM')
p.line(x=x, y=y3, line_width=2, color=Spectral10[2])

p.circle(x=x, y=y4, size=20, color=Spectral10[3], alpha=0.5, line_color='black', legend_label='10 uM')
p.line(x=x, y=y4, line_width=2, color=Spectral10[3])

p.legend.location = "bottom_right"
p.xaxis.axis_label = 'Time [min]'
p.yaxis.axis_label = 'Mean Intensity'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.major_label_text_font_size = '14pt'
p.legend.label_text_font_size = "14pt"

# show the results
show(p)

## Bar plots

In [10]:
output_notebook()

# Categories you want to plot
category1 = list(df_melt['Concentration'].unique())
category2 = list(df_melt['Pulse Duration'].unique())

# To create a list with all the combinations: [(500 uM, 5 min), (500 uM, 10 min),...,(10 uM, 30 min)]
x = [ (cat1, cat2) for cat1 in category1 for cat2 in category2 ]

# The values for each category combination: 500 uM for 5 mins is 2.3 mean fluorescent intensity
counts = df_melt['Result'] 

# Map the name of each catetory to the corresponding value 
source = ColumnDataSource(data=dict(x=x, counts=counts))

# Create the figure
p = figure(x_range=FactorRange(*x), plot_height=250, title="Mean fluorescent intensity during titration",
           toolbar_location=None, tools="")

# Choose the renderer: in this case, a vertical bar plot
p.vbar(x='x', top='counts', width=0.9, source=source, line_color="black",
       fill_color=factor_cmap('x', palette=Spectral10, factors=category2, start=1))

# Plot axis parameters
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.yaxis.axis_label = 'Mean Intensity'
p.xaxis.axis_label_text_font_size = '14pt'
p.yaxis.axis_label_text_font_size = '14pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.major_label_text_font_size = '14pt'

# Show the plot
show(p)

Loading BokehJS ...

## Histogram

In [11]:
# Where to output the figure
output_notebook()

# To make a histogram, you need to use numpy to calculate it
hist, edges = np.histogram(df_melt['Result'], density=True, bins=7)

# Create the figure
p = figure(width=400, plot_height=300)

# Renderers
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
     fill_color=Spectral10[9], line_color="black")

# Plotting options
p.xaxis.axis_label = 'Intensities'
p.yaxis.axis_label = 'Prob(Intensities)'

# Plotting options
p.xaxis.axis_label_text_font_size = '14pt'
p.yaxis.axis_label_text_font_size = '14pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.major_label_text_font_size = '14pt'

# Show figure
show(p)

Loading BokehJS ...

## Boxplot

To create a boxplot we first need to get the median and the quartiles for each of the groups we want to analyze. For this case, lets group the data as before, according to the concentration, and we check for each groups the median and the quartiles. Our groupby object (according to the concentration category) was previously saved as ``g1``. This is the object we will be using to create the box plot.

This box plot was inspired by an example figure from bokeh gallery. Visit this [link](https://docs.bokeh.org/en/0.10.0/docs/gallery/boxplot.html) to check out the plot. 

In [12]:
# Find the quartiles and IQR foor each category
q1 = g1.quantile(q=0.25)
q2 = g1.quantile(q=0.5)
q3 = g1.quantile(q=0.75)

iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# Categories you want to plot
categories = list(g1.groups.keys())

# If no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = g1.quantile(q=0.00)
qmax = g1.quantile(q=1.00)
upper.Result = [min([x,y]) for (x,y) in zip(list(qmax.iloc[:,0]),upper.Result) ]
lower.Result = [max([x,y]) for (x,y) in zip(list(qmin.iloc[:,0]),lower.Result) ]

# Where to output the figure
output_notebook()

# Create the figure
p = figure(tools="save", title="", x_range=categories, width=500, plot_height=300)

# stems
p.segment(categories, upper.Result, categories, q3.Result, line_width=2, line_color="black")
p.segment(categories, lower.Result, categories, q1.Result, line_width=2, line_color="black")

# boxes
p.rect(categories, (q3.Result+q2.Result)/2, 0.7, q3.Result-q2.Result,
    fill_color=Spectral10[0], line_width=2, line_color="black")
p.rect(categories, (q2.Result+q1.Result)/2, 0.7, q2.Result-q1.Result,
    fill_color=Spectral10[6], line_width=2, line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(categories, lower.Result, 0.2, 0.01, line_color="black")
p.rect(categories, upper.Result, 0.2, 0.01, line_color="black")

# Figure options
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "lightgray"
p.grid.grid_line_width = 2
p.yaxis.axis_label = 'Intensity'
p.xaxis.axis_label = 'Concentrations'
p.xaxis.axis_label_text_font_size = '14pt'
p.yaxis.axis_label_text_font_size = '14pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.major_label_text_font_size = '14pt'

# Show the figure
show(p)

Loading BokehJS ...

## Using Covid-19 Ireland cases dataset

Data can be downloaded from [Zenodo](https://zenodo.org/record/3901250#.XykEDi17FZI) by Frank Moriarty and Ciaran Prendergast. 

This data-set reports the number of diagnoses with coronavirus disease (COVID-19) as reported by the Department of Health in Ireland. This includes new cases diagnosed per day and cumulative cases, hospitalisations, ICU admissions, deaths, number of healthcare workers, number of clusters, gender of cases, age groups of cases, mode of transmission, age groups of those hospitalised, and cases per county.

### Prepare the data

In [13]:
df = pd.read_csv('Data/doh_covid_ie_cases_analysis.csv')
df.head()

,date,cases_cumul,cases_hosp,cases_icu,deaths_cumul,cases_hcp,clusters_cumul,cases_female,cases_male,cases_gender_unknown,...,cases_meath,cases_monaghan,cases_offaly,cases_roscommon,cases_sligo,cases_tipperary,cases_waterford,cases_westmeath,cases_wexford,cases_wicklow
0,16-Mar,271,84,6,2,59,23,NaN,NaN,NaN,...,<=5,0,<=5,<=5,<=5,<=5,7,7,<=5,9
1,17-Mar,350,108,7,2,84,26,NaN,NaN,NaN,...,6,0,<=5,<=5,<=5,<=5,7,7,<=5,13
2,18-Mar,438,140,12,3,114,27,189.0,241.0,7.0,...,11,0,<=5,<=5,<=5,<=5,7,12,<=5,15
3,19-Mar,584,173,13,3,147,29,257.0,320.0,7.0,...,10,0,7,<=5,<=5,<=5,8,14,<=5,17
4,20-Mar,712,211,17,4,159,35,311.0,393.0,8.0,...,11,<=5,9,<=5,<=5,8,9,16,<=5,22


To better visualize all the possible headers you have, it is useful to print them using ``df.keys()``. This way you will be able to see all the variables you have and think of the analysis and plots you would like to produce. 

In [14]:
df.keys()

Index(['date', 'cases_cumul', 'cases_hosp', 'cases_icu', 'deaths_cumul',
       'cases_hcp', 'clusters_cumul', 'cases_female', 'cases_male',
       'cases_gender_unknown', 'cases_below_1', 'cases_1_4', 'cases_under_5',
       'cases_5_14', 'cases_15_24', 'cases_25_34', 'cases_35_44',
       'cases_45_54', 'cases_55_64', 'cases_65_plus', 'cases_65_74',
       'cases_75_84', 'cases_85_plus', 'cases_age_unknown',
       'percent_comm_trns', 'percent_contact', 'percent_travel',
       'cases_hcp_travel', 'cases_hcp_no_travel', 'cases_hcp_tbc',
       'hosp_under_5', 'hosp_5_14', 'hosp_15_24', 'hosp_25_34', 'hosp_35_44',
       'hosp_45_54', 'hosp_55_64', 'hosp_65_plus', 'hosp_65_74', 'hosp_75_84',
       'hosp_85_plus', 'hosp_age_unknown', 'cases_carlow', 'cases_cavan',
       'cases_clare', 'cases_cork', 'cases_donegal', 'cases_dublin',
       'cases_galway', 'cases_kerry', 'cases_kildare', 'cases_kilkenny',
       'cases_laois', 'cases_leitrim', 'cases_limerick', 'cases_longford',
      

### Create a simple scatter plot

Now that we have prepared some data, let's create a simple scatter plot where we can visualize the data according to a chosen category, in this case let's have a look at the gender. We will plot the number of cases per gender versus the total number of cases. 

In [20]:
# Columns we want to plot
x = 'cases_cumul'
y1 = 'cases_female'
y2 = 'cases_male'
y3 = 'cases_gender_unknown'

# Where do you want to output the figure
output_notebook()

# Create the figure
p = figure(width=600, plot_height=300)

# Choose the renderr
p.line(x=x, y=y1, source=df, color='black', line_width=2)
p.line(x=x, y=y2, source=df, color='black', line_width=2)
p.line(x=x, y=y3, source=df, color='black', line_width=2)

p.circle(x=x, y=y1, source=df, legend_label="Females", fill_color=Spectral3[0], line_color='black', size=10)
p.circle(x=x, y=y2, source=df, legend_label="Males", fill_color=Spectral3[1], line_color='black', size=10)
p.circle(x=x, y=y3, source=df, legend_label="Unknown", fill_color=Spectral3[2], line_color='black', size=10)

# Adapt the axis
p.legend.location = "top_left"
p.xaxis.axis_label = 'Total number of cases'
p.yaxis.axis_label = 'Cases by gender'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.major_label_text_font_size = '14pt'
p.legend.label_text_font_size = "14pt"

# show the results
show(p)

Loading BokehJS ...

This data is already ordered temporaly, so we can observe in this plot that as time pases by, the number of cases is increasing and the number of female cases is actually higher than the number of male cases. 

You can see some of the tools that we defined in the ``figure()`` are: pan (to move around in the plot), box zoom (to zoom in and out of the plot), reset (to go back to the initial plotting seetings), save  (saves as a .png file). 

### Bar Plot

For this plot we will be only using the columns which represent the age values. For this, we will create a new DataFrame with the numbre of cases with the age groups.

In [21]:
df_ages = pd.melt(df,value_vars=['cases_15_24','cases_25_34','cases_35_44',\
             'cases_45_54','cases_55_64','cases_65_74','cases_75_84',\
             'cases_85_plus'], var_name='Ages', value_name='Result')
df_ages.head()

,Ages,Result
0,cases_15_24,28.0
1,cases_15_24,38.0
2,cases_15_24,47.0
3,cases_15_24,56.0
4,cases_15_24,63.0


We will make a box plot with the mean values for each age groups. For this, we create a new DataFrame where we compute the mean for all of the groups as we previously saw in the book chapter.

In [22]:
df_ages_sum = df_ages.groupby('Ages')['Result'].sum().reset_index()
df_ages_sum

,Ages,Result
0,cases_15_24,100925.0
1,cases_25_34,239697.0
2,cases_35_44,253143.0
3,cases_45_54,262691.0
4,cases_55_64,190077.0
5,cases_65_74,64766.0
6,cases_75_84,82489.0
7,cases_85_plus,85198.0


In [24]:
# Select the columns you want to plot
x = 'Ages'
y = 'Result'

# How do you want to output the figure
output_notebook()

# Create the figure
p = figure(plot_width=600, plot_height=250, y_axis_label='Age groups', x_range=df_ages_sum['Ages'].unique())

# Select the renderer
p.vbar(x=x, top=y, source=df_ages_sum, width=0.9, legend_label='Ages',\
       fill_color=factor_cmap('Ages', palette=Spectral8, factors=df_ages_sum['Ages'].unique()))

# Figure parameters
p.yaxis.axis_label = 'Number of cases'
p.xaxis.axis_label = 'Age groups'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '0pt'
p.yaxis.major_label_text_font_size = '14pt'
p.axis.minor_tick_line_color = None
p.add_layout(p.legend[0], 'right')

show(p)

Loading BokehJS ...

## Area Stack Plot

In [26]:
# Define a function to convert the date data into Day-Month-Year
def data_trans(x):
    months = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
              'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,}
    day,month = x.split('-')
    day = int(day)
    month = months[month]
    year = 2020
    return dt(year,month,day)

# Add a new column with new date time data (we used Transformation operation because we apply a function
# to all the values of the Column)
df['date_new'] = df['date'].transform(data_trans)

# Where to output the notebook
output_notebook()

# Generate the figure
p = figure(width=600,
           plot_height=300,
           x_axis_type='datetime')

# Defining the categories which will be usefd for plotting
names = ['cases_hosp','cases_icu','deaths_cumul','cases_hcp','clusters_cumul']

# We will use different names for the legends, which we define here
names_legends = ['Hosp', 'ICU', 'Total', 'HCP', 'Cluster']

# Choose the render 
p.varea_stack(stackers=names, x='date_new', color=brewer['Spectral'][len(names)], source=df, legend_label=names_legends)

# Where to place the legend
p.legend.location="top_left"

# X Axis parameters: we use the DatetimeTickFormatter to go from  2020-03-16 to 16 March 2020 
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

# For better visualization we orient the x ticks with 45 degrees
p.xaxis.major_label_orientation = 45

# Show the figure
show(p)

Loading BokehJS ...